In [5]:
from full_prod import *
from buchi import buchi_from_ltl
# from Visualize import plot_buchi
import numpy as np
from env_dynamic_ap import CurrentWorld
import scipy
# from plot_path_for_prod import *
from graphviz import Source

/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
r4 = Region((0,2),['basket'],'r4')
r5 = Region((1,2),['rball'],'r5')
r6 = Region((2,2),[],'r6')
c1 = Region((0,1),[],'c1')
c2 = Region((1,1),[],'c2')
c3 = Region((2,1),[],'c3')
r1 = Region((0,0),[],'r1')
r2 = Region((1,0),['basket'],'r2')
r3 = Region((2,0),['gball'],'r3')

region_list = [r1, r2, r3, r4, r5, r6, c1, c2, c3]

wfts = wFTS()

for i in region_list:
    wfts.add_states(i)
    
wfts.add_transition(r1,c1,1)
wfts.add_transition(c1,c2,1)
wfts.add_transition(c1,r4,1)
wfts.add_transition(c2,c3,1)
wfts.add_transition(c2,r2,1)
wfts.add_transition(c2,r5,1)
wfts.add_transition(c3,r6,1)
wfts.add_transition(c3,r3,1)

wfts.add_initial(r1)

formula = '<>(rball && <>basket) && <>[]r1'
buchi = buchi_from_ltl(formula,None)
my_buchi = Buchi_Automaton(buchi)

full_prod = FullProd(wfts,my_buchi)
full_prod.construct_fullproduct()
count = 0
for i in full_prod.states:
    for j in full_prod.transition[i].keys():
        if full_prod.transition[i][j] is not None:
            count += 1 

opt=search_opt_run(full_prod)
opt_path = [opt[0][i][0].coord for i in range(len(opt[0]))]
print 'Plan synthesized:'+str(opt_path)

Plan synthesized:[(0, 0), (0, 1), (1, 1), (1, 2), (1, 1), (1, 0), (1, 1), (0, 1), (0, 0), (0, 0), (0, 0), (0, 0)]


In [6]:
# Example Used in DQN Paper

ltl = "<>(A && <>(B && <>T)) && []<>A && []<>B && []!C".lower()
env = CurrentWorld(ltl)

region_list = ["r"+str(i) for i in range(100)]
for i in range(len(region_list)):
    coord = np.unravel_index(i, (10,10))
    if len(env.coord_dict[coord]) >0:
        region_list[i] = Region(coord, [ap.lower() for ap in env.coord_dict[coord]], region_list[i])
    else:
        region_list[i] = Region(coord, [], region_list[i])
    

wfts = wFTS(set(), {}, set())
for i in region_list:
    wfts.add_states(i)
    
for i in region_list:
    current_coord = list(i.coord)
    candidates = [np.array(current_coord), np.add(current_coord,[0,1]), np.add(current_coord,[0,-1]), 
                  np.add(current_coord,[1,0]), np.add(current_coord,[-1,0])]
    candidates = [np.ravel_multi_index(c, env.shape[:-1]) for c in candidates if c[0]>=0 and c[1]>=0 and c[0]<env.shape[0] and c[1]<env.shape[1]]
    for c in candidates:
        wfts.add_transition(i, region_list[c], 1)
            
wfts.add_initial(region_list[np.ravel_multi_index(env.start_coord, env.shape[:-1])])
    

buchi = buchi_from_ltl(ltl,None)
my_buchi = Buchi_Automaton(buchi)

# rabin = Rabin_Automaton(ltl, env.coord_dict)

full_prod = FullProd(wfts, my_buchi)
full_prod.construct_fullproduct()
count = 0
for i in full_prod.states:
    for j in full_prod.transition[i].keys():
        if full_prod.transition[i][j] is not None:
            count += 1 

opt=search_opt_run(full_prod)
opt_path = [opt[0][i][0].coord for i in range(len(opt[0]))]
print 'Plan synthesized:'+str(opt_path)

0
0
Plan synthesized:[(4, 1), (3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (2, 5), (2, 6), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (7, 6), (7, 5), (6, 5), (6, 4), (6, 3), (6, 2), (5, 2), (6, 2), (7, 2), (7, 3), (7, 4), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 7), (7, 7), (6, 7), (5, 7), (4, 7), (3, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (7, 6), (7, 5), (6, 5), (6, 4), (5, 4), (5, 3), (5, 2), (6, 2), (6, 2), (6, 3), (6, 3), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (6, 7), (5, 7), (4, 7), (3, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (7, 6), (7, 5), (6, 5), (6, 4), (5, 4), (5, 3), (5, 2), (6, 2)]


In [3]:
#plot buchi
my_buchi.plot()

#plot full product
full_prod_graph = full_prod.return_graph()
plot(full_prod_graph,'full_prod')

Graph Successfully Plotted
Graph Successfully Plotted
